In [ ]:
%matplotlib inline

In [1]:

import datetime as dt

import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("ticks")

# Largest Context:
#sns.set_context("talk")

from metpy.calc import dewpoint_from_relative_humidity
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo
from metpy.units import units

import os

# Height of the station to calculate MSLP
hgt_example = 206.

def calc_mslp(t, p, h):
    return p * (1 - (0.0065 * h) / (t + 0.0065 * h + 273.15)) ** (-5.257)


testdata = pd.read_csv('/data/accp/a/snesbitt/scamp/logger/SCAMPCR6_SCAMP_1min.dat',header=1).drop([0,1])
testdata2 = pd.read_csv('/data/accp/a/snesbitt/scamp/logger/SCAMPCR6_SCAMP_1min_nogust.dat',header=1).drop([0,1])
testdata3 = pd.read_csv('/data/accp/a/snesbitt/scamp-2023/LoggerNet/SCAMPCR6_SCAMP_2022_1min.dat',header=1).drop([0,1])

testdata_all = pd.concat([testdata,testdata2,testdata3]).replace('NAN',np.nan)

testdata_all['Time_Eastern'] = pd.to_datetime(testdata_all['TIMESTAMP'])+pd.Timedelta(value=1, unit='hours')
# testdata_all=testdata_all.set_index('Time_Eastern')

timestamps_est = np.array(testdata_all.Time_Eastern, dtype=dt.datetime)
temp_times = np.zeros(len(timestamps_est),dtype=dt.datetime)
for i in range(len(timestamps_est)):
    temp_times[i] = timestamps_est[i]+pd.Timedelta(value=5, unit='hours')

testdata_all['UTC'] = temp_times
testdata_all=testdata_all.set_index('UTC')



/tmp/ipykernel_29691/3923200329.py:27: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  testdata = pd.read_csv('/data/accp/a/snesbitt/scamp/logger/SCAMPCR6_SCAMP_1min.dat',header=1).drop([0,1])
/tmp/ipykernel_29691/3923200329.py:29: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  testdata3 = pd.read_csv('/data/accp/a/snesbitt/scamp-2023/LoggerNet/SCAMPCR6_SCAMP_2022_1min.dat',header=1).drop([0,1])


In [ ]:

dates = pd.date_range(start='2022-11-18',end='2022-11-18',freq='1D')

for idate in dates:
  
    t1=idate
    t2=idate+pd.Timedelta(1, unit='D')
    mask = (testdata_all.index >= t1) & (testdata_all.index <= t2)
    print(t1, t2, mask.sum())

    if mask.sum() > 0:

        testdata = testdata_all.loc[mask]

        # Temporary variables for ease
        temp = pd.to_numeric(testdata['Air_Temp_Avg'],errors='coerce')
        pressure = pd.to_numeric(testdata['Abs_Pressure_Avg'],errors='coerce')
        rh = pd.to_numeric(testdata['Rel_Hum_Avg'],errors='coerce')
        ws = pd.to_numeric(testdata['Wspd_Avg'],errors='coerce')
        ws[ws>50] == np.nan
        wsmax = pd.to_numeric(testdata['Wspd_Max'],errors='coerce')
        wsmax[wsmax > 50] = np.nan
        wd = pd.to_numeric(testdata['Wdir_Vct'],errors='coerce')
        precip = pd.to_numeric(testdata['Current_Depth'],errors='coerce')
        precip_rate = np.diff(pd.to_numeric(testdata['Current_Depth'],errors='coerce'))
        precip_rate[precip_rate < 0.05] = 0.
        precip_rate = np.concatenate([[0],precip_rate],axis=0)
        precip_daily = np.cumsum(precip_rate)
        freq = pd.to_numeric(testdata['Geonor_Freq'],errors='coerce')
        date = pd.to_datetime(testdata.index) #+ dt.timedelta(hours=1)


        data = pd.DataFrame({'Wind speed (m/s)': (np.array(ws) * units('m/s')),
                'Wind gust (1 sec, m/s)': (np.array(wsmax) * units('m/s')),
                'Wind direction (deg)': np.array(wd) * units('degrees'),
                'Dewpoint (F)': dewpoint_from_relative_humidity((np.array(temp) * units.degC),
                                                            np.array(rh) / 100.),
                'Pressure (hPa)': np.array(pressure) * units('hPa'),
                'Temperature (F)': (np.array(temp) * units('degC')),
                'Sea level pressure (hPa)': calc_mslp(np.array(temp), np.array(pressure), hgt_example) * units('hPa'),
                'Relative humidity': np.array(rh),
                'Precipitation rate (mm/hr)': np.array(precip_rate*60)* units('mm'),
                'Precip accumulation (mm)': np.array(precip_daily)* units('mm'), 'times': np.array(date),
                'Frequency': np.array(freq)})
        data.index=date

        fig = plt.figure(figsize=(13,8.5))

        ax=fig.subplots(nrows=5,ncols=1,sharex=True)

        data['Temperature (F)'].plot(ax=ax[0])
        data['Dewpoint (F)'].plot(ax=ax[0])
        ax[0].set_ylabel('Temp, Dewpt\n(°C)')

        data['Wind speed (m/s)'].plot(ax=ax[1])
        data['Wind gust (1 sec, m/s)'].plot(ax=ax[1])
        if np.isfinite(np.max(data['Wind gust (1 sec, m/s)'])):
            ax[1].set_ylim([0,np.max(data['Wind gust (1 sec, m/s)'])])
        else:
            ax[1].set_ylim([0,np.max(data['Wind speed (m/s)'])])
        ax[1].set_ylabel('Wind speed,\ngust (m/s)')

        data['Wind direction (deg)'].plot(ax=ax[2],color='g')
        ax[2].set_ylim([0,360])
        ax[2].set_ylabel('Wind dir\n(deg)')

        data['Sea level pressure (hPa)'].plot(ax=ax[3])
        ax[3].set_ylabel('Sea level\npressure (hPa)')
        ax[3].set_ylim([980,1020])

        data['Precip accumulation (mm)'].plot(ax=ax[4])
        ax[4].set_ylabel('Geonor\nPrecip (mm)')
        #data['Precipitation rate (mm/hr)'].plot(ax=ax, secondary_y=True)

        fig.suptitle('UIUC SCAMP weather instruments: '+t1.strftime('%Y%m%d %H:%M')+' to '+t2.strftime('%Y%m%d %H:%M'), y=0.90)

        #plt.savefig('/data/accp/a/snesbitt/scamp/logger/images2/'+t1.strftime('%Y%m%d_%H:%M_SCAMPwxstn'+'.png'),dpi=150)

        plt.close()

In [ ]:
fig = plt.figure(figsize=(13,8.5))

ax=fig.subplots(nrows=5,ncols=1,sharex=True)

data['Temperature (F)'].plot(ax=ax[0])
data['Dewpoint (F)'].plot(ax=ax[0])
ax[0].set_ylabel('Temp, Dewpt\n(°C)')

data['Wind speed (m/s)'].plot(ax=ax[1])
data['Wind gust (1 sec, m/s)'].plot(ax=ax[1])
#ax[1].set_ylim([0,np.max(data['Wind gust (1 sec, m/s)'])])
ax[1].set_ylabel('Wind speed,\ngust (m/s)')

data['Wind direction (deg)'].plot(ax=ax[2],color='g')
ax[2].set_ylim([0,360])
ax[2].set_ylabel('Wind dir\n(deg)')

data['Sea level pressure (hPa)'].plot(ax=ax[3])
ax[3].set_ylabel('Sea level\npressure (hPa)')

data['Precip accumulation (mm)'].plot(ax=ax[4])
ax[4].set_ylabel('Geonor\nPrecip (mm)')
#data['Precipitation rate (mm/hr)'].plot(ax=ax, secondary_y=True)

fig.suptitle('UIUC SCAMP weather instruments: '+t1.strftime('%Y%m%d %H:%M')+' to '+t2.strftime('%Y%m%d %H:%M'), y=0.90)

In [ ]:
timestamps_est = np.array(testdata_all.TIMESTAMP)
print(type(timestamps_est[0]))
pd.to_datetime(testdata_all.insert(15, 'UTC',''))
#testdata_all['UTC'] = np.zeros(len(timestamps_bad))
temp_times = np.zeros(len(timestamps_est),dtype=dt.datetime)
for i in range(len(timestamps_est)):
    temp_times[i] = dt.datetime.strptime(timestamps_est[i],'%Y-%m-%d %H:%M:%S')
    #testdata_all.UTC[i] = temp_times[i] + pd.Timedelta(6, unit='h')
    print(testdata_all.loc[testdata_all.TIMESTAMP == timestamps_est[i],'UTC'])
    testdata_all.loc[testdata_all.TIMESTAMP == timestamps_est[i],'UTC'] == temp_times[i]
    print(testdata_all.loc[testdata_all.TIMESTAMP == timestamps_est[i],'UTC'] == temp_times[i])

In [ ]:
#USE
timestamps_est = np.array(testdata_all.Time_Eastern, dtype=dt.datetime)
temp_times = np.zeros(len(timestamps_est),dtype=dt.datetime)
for i in range(len(timestamps_est)):
    temp_times[i] = timestamps_est[i]+pd.Timedelta(value=5, unit='hours')

testdata_all['UTC'] = temp_times

In [2]:
testdata_all

,TIMESTAMP,RECORD,PTemp_Avg,Batt_Volt_Avg,Air_Temp_Avg,Rel_Hum_Avg,Abs_Pressure_Avg,Wspd_Max,Wspd_Avg,Wdir_Vct,Geonor_Freq,Geonor_Depth_Raw,Current_Depth,Depth_1min,Precip_Rate,Time_Eastern
UTC,,,,,,,,,,,,,,,,
2021-12-08 16:46:00,2021-12-08 10:46:00,0,3.218,13.36,-2.7,74.1,988.4,4.1,2.4,218.4,1505.914,93.41496,93.41496,93.41496,5604.898,2021-12-08 11:46:00
2021-12-08 16:47:00,2021-12-08 10:47:00,1,3.218,13.36,-2.6,74.1,988.4,3.5,2.5,244.4,1505.955,93.42503,93.42503,0,0,2021-12-08 11:47:00
2021-12-08 16:48:00,2021-12-08 10:48:00,2,3.225,13.36,-2.6,73.4,988.4,2.9,1.9,227.8,1505.959,93.42615,93.42615,0,0,2021-12-08 11:48:00
2021-12-08 16:49:00,2021-12-08 10:49:00,3,3.235,13.36,-2.7,73.6,988.5,4.2,3,205.2,1505.97,93.42876,93.42876,0,0,2021-12-08 11:49:00
2021-12-08 16:50:00,2021-12-08 10:50:00,4,3.245,13.36,-2.5,73.9,988.4,3.8,3.2,206.9,1505.948,93.42336,93.42336,0,0,2021-12-08 11:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-04 15:42:00,2023-03-04 09:42:00,24164,4.756,13.35,-0.1,89.0,981,3.0,1.3,50.4,2537.72,447.4625,447.4625,0,0,2023-03-04 10:42:00
2023-03-04 15:43:00,2023-03-04 09:43:00,24165,4.764,13.35,-0.1,89.1,981,1.8,1.0,45.7,2537.821,447.5069,447.5069,0,0,2023-03-04 10:43:00
2023-03-04 15:44:00,2023-03-04 09:44:00,24166,4.772,13.35,-0.2,89.1,981,2.4,0.5,35.0,2537.882,447.5334,447.5334,0,0,2023-03-04 10:44:00


In [ ]:
times_est = np.array(testdata_all.index)
testdata_all.insert(15, 'UTC','')
#temp_times = np.zeros(len(times_est),dtype=dt.datetime)
for i in range(len(times_est)):
    #temp_times[i] = times_est[i]
    #testdata_all.UTC[i] = times_est[i]
    testdata_all.loc[testdata_all.index == times_est[i],'UTC'] == times_est[i]
    print(testdata_all.loc[testdata_all.index == times_est[i],'UTC'])
    testdata_all.UTC[i] = testdata_all.UTC[i] + pd.Timedelta(5, unit='h')
    print(testdata_all.UTC[i])
    #print(testdata_all.loc[testdata_all.TIMESTAMP == timestamps_est[i],'UTC'])

In [ ]:
dates = pd.date_range(start='2022-11-18',end='2022-11-18',freq='1D')

for idate in dates:
  
    t1=idate
    t2=idate+pd.Timedelta(1, unit='D')
    mask = (testdata_all.index >= t1) & (testdata_all.index <= t2)
    print(t1, t2, mask.sum())

testdata = testdata_all.loc[mask]
